# ¿Monoides pa' qué? Parte 2

1. Paralelismo
2. Reducciones (calcular propiedades de colecciones)
3. Los monoides son componibles: si `a` y `b` son monoides, `(a,b)` es un monoide.

In [1]:
import Data.Monoid (Sum (..), Product (..), mconcat)

mconcat [(Product 5, Sum 5), (Product (-4), Sum (-4)), (Product 3, Sum 3)]

(Product {getProduct = -60},Sum {getSum = 4})

Crea una función que dado un número entero positivo `n`, te regresa la tupla `(Product x, Sum y)` donde `x` es el producto de los primeros `n` números, y `y` es la suma de los primeros `n` números.

In [8]:
productoYSuma :: Int -> (Product Int, Sum Int)
productoYSuma n = mconcat [(Product x, Sum x) | x <- [1..n]]

productoSuma :: Int -> (Int, Int)
productoSuma n = foldr (\(x, y) (x', y') -> (x*x', y+y')) (1, 0) $ map (\x -> (x, x)) [1..n]

Line 5: Fuse foldr/map
Found:
foldr (\ (x, y) (x', y') -> (x * x', y + y')) (1, 0)
  $ map (\ x -> (x, x)) [1 .. n]
Why not:
foldr
  ((\ (x, y) (x', y') -> (x * x', y + y')) . (\ x -> (x, x))) (1, 0)
  [1 .. n]

In [9]:
productoYSuma 10

productoSuma 10

(Product {getProduct = 3628800},Sum {getSum = 55})

(3628800,55)

## Exponenciación

In [10]:
import Data.Semigroup (mtimesDefault)

elevar :: Num a => a -> Int -> a
elevar x y = getProduct $ mtimesDefault y (Product x)

In [12]:
import qualified Data.Matrix as M
M.fromLists [[0, 1], [1, 1]]

┌     ┐
│ 0 1 │
│ 1 1 │
└     ┘

In [13]:
:i M.Matrix

type M.Matrix :: * -> *
data M.Matrix a
  = Data.Matrix.M {M.nrows :: {-# UNPACK #-}Int,
                   M.ncols :: {-# UNPACK #-}Int,
                   Data.Matrix.rowOffset :: {-# UNPACK #-}Int,
                   Data.Matrix.colOffset :: {-# UNPACK #-}Int,
                   Data.Matrix.vcols :: {-# UNPACK #-}Int,
                   Data.Matrix.mvect :: Data.Vector.Vector a}
  	-- Defined in ‘Data.Matrix’
instance Show a => Show (M.Matrix a) -- Defined in ‘Data.Matrix’
instance Eq a => Eq (M.Matrix a) -- Defined in ‘Data.Matrix’
instance Num a => Num (M.Matrix a) -- Defined in ‘Data.Matrix’
instance Applicative M.Matrix -- Defined in ‘Data.Matrix’
instance Functor M.Matrix -- Defined in ‘Data.Matrix’
instance Monoid a => Monoid (M.Matrix a) -- Defined in ‘Data.Matrix’
instance Monoid a => Semigroup (M.Matrix a) -- Defined in ‘Data.Matrix’
instance Foldable M.Matrix -- Defined in ‘Data.Matrix’
instance Traversable M.Matrix -- Defined in ‘Data.Matrix’

In [14]:
import qualified Data.Matrix as M

elevar (M.fromLists [[0, 1], [1, 1]]) 10
elevar 5 5

┌       ┐
│ 34 55 │
│ 55 89 │
└       ┘

3125

In [15]:
fib :: Int -> Int
fib = M.getElem 1 1 . elevar fibMatrix
    where fibMatrix = M.fromLists [[0, 1], [1, 1]]

In [16]:
fib 10

34

## Validación

In [17]:
import Data.Char (isLower, isUpper, isDigit)

In [18]:
longitudValida :: String -> Bool
longitudValida = (>=8) . length

contieneMayusculas :: String -> Bool
contieneMayusculas = any isUpper

contieneMinusculas :: String -> Bool
contieneMinusculas = any isLower

contieneNumeros :: String -> Bool
contieneNumeros = any isDigit

In [19]:
longitudValida "Hola Mundo"
contieneMayusculas "Hola Mundo"
contieneMinusculas "Hola Mundo"
contieneNumeros "Hola Mundo9"

True

True

True

True

In [20]:
:i ->

type (->) :: * -> * -> *
data (->) a b
  	-- Defined in ‘GHC.Prim’
infixr -1 ->
instance Applicative ((->) r) -- Defined in ‘GHC.Base’
instance Functor ((->) r) -- Defined in ‘GHC.Base’
instance Monad ((->) r) -- Defined in ‘GHC.Base’
instance Monoid b => Monoid (a -> b) -- Defined in ‘GHC.Base’
instance Semigroup b => Semigroup (a -> b) -- Defined in ‘GHC.Base’

In [50]:
import Data.Semigroup (All (..))

valida :: String -> Bool
valida = getAll . foldMap (All .) validaciones
    where validaciones =  [longitudValida, contieneMayusculas, contieneMinusculas, contieneNumeros]

In [22]:
import Data.Semigroup (All (..))
:t mconcat [All . longitudValida, All . contieneMayusculas, All . contieneMinusculas, All . contieneNumeros]

mconcat [All . longitudValida, All . contieneMayusculas, All . contieneMinusculas, All . contieneNumeros] :: String -> All

In [52]:
valida "HolaMundo"

False

## Reducibles (Foldable)

Un reducible es un tipo de dato que puede ir acumulándose, un valor a la vez, hasta llegar a un solo resultado.

In [40]:
import Data.Semigroup (Min (..), Max (..))

In [41]:
:i Foldable

type Foldable :: (* -> *) -> Constraint
class Foldable t where
  Data.Foldable.fold :: Monoid m => t m -> m
  foldMap :: Monoid m => (a -> m) -> t a -> m
  Data.Foldable.foldMap' :: Monoid m => (a -> m) -> t a -> m
  foldr :: (a -> b -> b) -> b -> t a -> b
  Data.Foldable.foldr' :: (a -> b -> b) -> b -> t a -> b
  foldl :: (b -> a -> b) -> b -> t a -> b
  Data.Foldable.foldl' :: (b -> a -> b) -> b -> t a -> b
  foldr1 :: (a -> a -> a) -> t a -> a
  foldl1 :: (a -> a -> a) -> t a -> a
  Data.Foldable.toList :: t a -> [a]
  null :: t a -> Bool
  length :: t a -> Int
  elem :: Eq a => a -> t a -> Bool
  maximum :: Ord a => t a -> a
  minimum :: Ord a => t a -> a
  sum :: Num a => t a -> a
  product :: Num a => t a -> a
  {-# MINIMAL foldMap | foldr #-}
  	-- Defined in ‘Data.Foldable’
instance Foldable M.Matrix -- Defined in ‘Data.Matrix’
instance Foldable Min -- Defined in ‘Data.Semigroup’
instance Foldable Max -- Defined in ‘Data.Semigroup’
instance Foldable [] -- Defined in ‘Data.Foldable’
instance Foldable Sum -- Defined in ‘Data.Foldable’
instance Foldable Product -- Defined in ‘Data.Foldable’
instance Foldable Maybe -- Defined in ‘Data.Foldable’
instance Foldable (Either a) -- Defined in ‘Data.Foldable’
instance Foldable ((,) a) -- Defined in ‘Data.Foldable’

In [42]:
:t foldMap

foldMap :: forall (t :: * -> *) m a. (Foldable t, Monoid m) => (a -> m) -> t a -> m

Si tenemos una colección que es reducible, y los elementos en la colección los podemos mapear a un monoide, tenemos una forma sencilla de calcular propiedades de la colección.

In [43]:
{-# LANGUAGE DeriveFoldable #-}
data Tree a = Leaf a
            | Node (Tree a) a (Tree a)
    deriving (Foldable, Show)

tree :: Tree Int
tree = Node (Leaf 4) 9 (Node (Leaf 42) 28 (Leaf 1))

foldMap Sum     tree
foldMap Product tree
foldMap (\x -> (Sum x, Product x)) tree
foldMap (\x -> (Min x, Max x)) tree

Sum {getSum = 84}

Product {getProduct = 42336}

(Sum {getSum = 84},Product {getProduct = 42336})

(Min {getMin = 1},Max {getMax = 42})

In [44]:
xs :: [Int]
xs = [5,100, 86, 57]

foldMap Sum     xs
foldMap Product xs
foldMap (\x -> (Sum x, Product x)) xs
foldMap (\x -> (Min x, Max x)) xs

Sum {getSum = 248}

Product {getProduct = 2451000}

(Sum {getSum = 248},Product {getProduct = 2451000})

(Min {getMin = 5},Max {getMax = 100})

Implementa la función `foldMap`.

In [45]:
:t foldr

foldr :: forall (t :: * -> *) a b. Foldable t => (a -> b -> b) -> b -> t a -> b

In [48]:
foldMap :: (Foldable t, Monoid m) => (a -> m) -> t a -> m
foldMap f = foldr ((<>) . f) mempty

In [49]:
data Tree a = Leaf a
            | Node (Tree a) a (Tree a)
    deriving (Foldable, Show)

tree :: Tree Int
tree = Node (Leaf 4) 9 (Node (Leaf 42) 28 (Leaf 1))

foldMap Sum     tree

Sum {getSum = 84}